In [1]:
import requests
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import undetected_chromedriver as uc
import locale

import calendar
import random

In [ ]:

months = {
    "ene": "Jan", "feb": "Feb", "mar": "Mar", "abr": "Apr",
    "may": "May", "jun": "Jun", "jul": "Jul", "ago": "Aug",
    "sept": "Sep", "oct": "Oct", "nov": "Nov", "dic": "Dec"
}

locale.setlocale(locale.LC_TIME, 'C')

json_pages_info = {"pages": []}

options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = uc.Chrome(options=options)

pages = 10

for page in range(0, pages):

	url = f'https://www.google.com/search?q=%22Estallido+social%22+site%3Aelpais.com&tbs=cdr:1,cd_min:11/15/2019,cd_max:12/17/2023&start={page * 10}'

	driver.get(url)

	time.sleep(2)

	articles_section = driver.find_element(By.CLASS_NAME, "dURPMd")
	articles = articles_section.find_elements(By.CLASS_NAME, "MjjYud")

	for article in articles:
		print("revisando artículo...")
		try: 
			originalDate = article.find_element(By.CLASS_NAME, "YrbPuc").find_element(By.TAG_NAME, "span").text
			for es, en in months.items():
				if es in originalDate:
						originalDate = originalDate.replace(es, en)
						break
				
			date_epoch = int(time.mktime(time.strptime(originalDate, "%d %b %Y")))

			if (date_epoch < 1573786800 or date_epoch > 1702782000):
				print("La noticia no corresponde a la fecha solicitada")
				continue

			# check if the article is from El Mostrador
			if (not ("https://elpais.com" in article.text)):
				print("El artículo no es de El Mostrador")
				continue

			try:
				title = article.find_element(By.TAG_NAME, "h3").text
				description = article.find_element(By.CLASS_NAME, "kb0PBd ").find_elements(By.TAG_NAME, "span")[1].text
				link = article.find_element(By.TAG_NAME, "a").get_attribute("href")

				link_info = {
					"newscast" : "El País",
					"title": title,
					"description": description,
					"category": "The site does not provide a category",
					"date": originalDate,
					"image_link": "not found initially",
					"author": "not found initially",
					"link": link,
				}

				print("información incluida!")
				json_pages_info["pages"].append(link_info)

			except Exception as e:
				print(f"Error al extraer información del artículo: {e}")
				continue

		except Exception as e:
			print(f"Error al procesar el artículo: {e}")
			continue

print("Almacenando información en el archivo JSON...")
with open(f"../archive/temp/el_pais_pages_08072025.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


with open(f"output.html", "w", encoding="utf-8") as f:
	f.write(driver.page_source)

driver.quit()

In [ ]:
import json

el_pais_data01 = {}
el_pais_data02 = {}

clean_data = {
	"pages": []
}

with open("../archive/temp/el_pais_pages_with_content_27_05_25.json", 'r', encoding='utf-8') as file:
	el_pais_data01 = json.load(file)

with open("../archive/temp/el_pais_pages_08072025_raw.json", 'r', encoding='utf-8') as file:
	el_pais_data02 = json.load(file)

coincidence = 0

all_data = el_pais_data01["pages"] + el_pais_data02["pages"]
all_links = [page["link"] for page in all_data]

clean_data = {
	"pages": []
}

clean_data_links = []

print("el pais 01 " + str(len(el_pais_data01["pages"])))
print("el pais 02 " + str(len(el_pais_data02["pages"])))
print("all data " + str(len(all_data)))

coincidence_boolean = False

for page in all_data:
	for link in clean_data_links:
		if page["link"] in link:
			print(f"Coincidencia encontrada: {page['link']} con link {link}")
			coincidence += 1
			coincidence_boolean = True
			break
	if not coincidence_boolean:
		print(f"Agregando página: {page['link']}")
		clean_data["pages"].append(page)
		clean_data_links.append(page["link"])
	coincidence_boolean = False


print(f"Total coincidences: {coincidence}")
print(f"Total unique links: {len(clean_data['pages'])}")

with open(f"../archive/temp/el_pais_cleanData_080725.json", 'w', encoding='utf-8') as file:
	json.dump(clean_data, file, ensure_ascii=False, indent=4)


## Extraction of content

In [ ]:

extracted_pages = {"pages": []}
extracted_pages_with_content = {"pages": []}
total_pages = 0
pages_succeeded = 0
pages_failed = 0
error_occurred = False
driver = None
ad_message = "Suscríbase aquí a la newsletter de EL PAÍS América y reciba todas las claves informativas de la actualidad de la región."
limit_of_pages = "all"
start_page = 0

content = ""
author = ""

print("[INFO] Loading extracted pages from JSON file...")
with open("../archive/pages_extracted/el_pais/el_pais_cleanData_080725.json", 'r', encoding='utf-8') as file:
		extracted_pages = json.load(file)
		print(f"[INFO] {len(extracted_pages['pages'])} pages loaded.")
		if limit_of_pages == "all":
			limit_of_pages = len(extracted_pages["pages"])


if start_page >= len(extracted_pages["pages"]):
		print(f"[ERROR] start_page is out of range. The maximum page index is {len(extracted_pages['pages']) - 1}.")
		raise SystemExit


def extract_from_script():
		print("[INFO] Trying to extract content from <script> tags...")
		data = {"content": "", "author": "", "raw_json_content": ""}

		scripts = driver.find_elements(By.XPATH, "//head/script")
		data_in_script = {}
		content_extracted_by_json = False
		for script in scripts: 
			script_content = script.get_attribute("innerHTML").strip()

			# Intentar interpretarlo como JSON
			is_json = False
			content_extracted_by_json = False
			try:
					data_in_script = json.loads(script_content)   # si es JSON válido, lo convierte en dict/list
					test = data_in_script["articleBody"]
					if test is not None:
						print("[INFO] Content found in <script> tag.")
					is_json = True
			except json.JSONDecodeError:
					is_json = False

			except Exception as e:
					is_json = False
			
			try:
				if is_json:
						data["content"] = data_in_script["articleBody"]
						data["raw_json_content"] = data_in_script

						try:
								data["author"] = data_in_script["author"][0]["name"]
						except Exception as e:
								print("[ERROR] A error occurred while extracting author name", flush=True)
								data["author"] = "not found"

						content_extracted_by_json = True
						print("[INFO] The content has been extracted. Method: JSON", flush=True)
						break
			except Exception as e:
					print(f"[ERROR] A error occurred while extracting content from JSON script: {e}", flush=True)
					content_extracted_by_json = False

		return content_extracted_by_json, data

def extract_from_DOM():
		print("[INFO] Trying to extract content from DOM...")
		content = ""
		author = ""

		content_container = driver.find_element(By.CLASS_NAME, "clearfix")
		paragraph = content_container.find_elements(By.TAG_NAME, "p") if content_container else None
		content = ""
		for p in paragraph:
			content += p.text.strip() + "\n"

		try:
				author = driver.find_element(By.CLASS_NAME, "a_md_a").text.strip()
		except:
				author = "not found"

		print("[INFO] The content has been extracted. Method: DOM", flush=True)
		return content, author

data_extracted = True
re = False

print("[INFO] Starting extraction process...")
for page_index, page in enumerate(extracted_pages["pages"]):

		if page_index < start_page:
				print("[INFO] Skipping page: ", page_index)
				continue
		if limit_of_pages == total_pages:
				print("[INFO] Limit of pages reached.")
				break
		if page["content"] != "The content has not been extracted yet":
				print(f"[INFO] Page {page_index} already has content. Skipping...")
				extracted_pages_with_content["pages"].append(page)
				continue

		total_pages += 1

		print(f"\n[INFO] Checking page: {page_index} | Link: {page['link']}" )
		url = page['link']
		first = True

		while (re or first):
			first = False
			
			try: 
				time.sleep(1)

				if (not re): 
					options = uc.ChromeOptions()
					options.add_argument("--no-sandbox")
					options.add_argument("--disable-blink-features=AutomationControlled")

					driver = uc.Chrome(options=options)

					driver.get(url)

				re = False
				time.sleep(random.uniform(6, 12))

				try: 
					content_extracted_by_json, data = extract_from_script()
					if not content_extracted_by_json: 
							content, author = extract_from_DOM()

							if author == "not found":
									try:
											author = driver.find_element(By.CLASS_NAME, "a_md_a").text.strip()
									except:
											author = "not found"
					else:
							content = data["content"]
							author = data["author"]
							page["raw_json_content"] = data["raw_json_content"] if data["raw_json_content"] != "" else "not found initially"

					# Clean up ad message
					content = content.replace(ad_message, "").strip()
				except Exception as e:
					print(f"[ERROR] A error occurred while extracting content", flush=True)
					raise

			except Exception as e:
				print(f"[ERROR] A error occurred while processing the page: {page_index}", flush=True)
				print(f"[ERROR] Site: {url}", flush=True)
				print(e, flush=True)
				response = input("Retry? y/n")
				if response.lower() == 'y':
					print("[INFO] Retry...")
					re = True
				else: 
					print("[INFO] Cancelling...")
					error_occurred = True
					re = False
			
			if (not re):
				driver.quit()

		if error_occurred:
			content = "A error occurred while extracting content"
			author = "A error occurred while extracting content"
			pages_failed += 1
			error_occurred = False
		else:
			pages_succeeded += 1

		page["content"] = content
		page["author"] = author
		extracted_pages_with_content["pages"].append(page)

		print("[INFO] Storing information in a JSON file...")
		with open(f"../archive/temp/pages_with_content/el_pais_cleanData_080725_with_content.json", 'w', encoding='utf-8') as file:
			json.dump(extracted_pages_with_content, file, ensure_ascii=False, indent=4)

		driver.quit()

		time.sleep(random.uniform(6, 12))

# ----------------- Stats -----------------
print("\n\n")
print("----- STATS -----")

try:
	print(f"Amount of pages processed: {total_pages}")
	print(f"% Pages succeeded [{pages_succeeded}]: {round(100 * pages_succeeded/total_pages, 2)}")
	print(f"% Pages failed [{pages_failed}]: {round(100 * pages_failed/total_pages, 2)}")
except: 
	print("A error occurred while processing the stats")

[INFO] Loading extracted pages from JSON file...
[INFO] 128 pages loaded.
[INFO] Starting extraction process...
[INFO] Page 0 already has content. Skipping...
[INFO] Page 1 already has content. Skipping...
[INFO] Page 2 already has content. Skipping...
[INFO] Page 3 already has content. Skipping...
[INFO] Page 4 already has content. Skipping...
[INFO] Page 5 already has content. Skipping...
[INFO] Page 6 already has content. Skipping...
[INFO] Page 7 already has content. Skipping...
[INFO] Page 8 already has content. Skipping...
[INFO] Page 9 already has content. Skipping...
[INFO] Page 10 already has content. Skipping...
[INFO] Page 11 already has content. Skipping...
[INFO] Page 12 already has content. Skipping...
[INFO] Page 13 already has content. Skipping...
[INFO] Page 14 already has content. Skipping...
[INFO] Page 15 already has content. Skipping...
[INFO] Page 16 already has content. Skipping...
[INFO] Page 17 already has content. Skipping...
[INFO] Page 18 already has content